In [19]:
from pyspark.sql import SparkSession
import pandas as pd
spark = SparkSession.builder.appName('adult_logReg').getOrCreate()
df = spark.read.csv('Dataset/Bank.csv', inferSchema = True, header=True)
df.show(3)
Pdf=df.toPandas()

+---+---------+-------+-----------+-------+-------+----+---------+-----+---+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+-------+
|age|      job|marital|  education|default|housing|loan|  contact|month|day|duration|campaign|pdays|previous|   poutcome|emp var rate|cons price idx|cons conf idx|euribor3m|nr employed|Deposit|
+---+---------+-------+-----------+-------+-------+----+---------+-----+---+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+-------+
| 56|housemaid|married|   basic.4y|     no|     no|  no|telephone|  may|mon|     261|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|     5191.0|     no|
| 57| services|married|high.school|unknown|     no|  no|telephone|  may|mon|     149|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|     5191.0|     no|
| 37| services|married|high.sc

In [20]:
cols = df.columns

In [21]:
df.drop(['job','marital','education','cons conf idx','housing'],axis=1)

TypeError: drop() got an unexpected keyword argument 'axis'

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler

categoricalColumns = ["workclass", "education", "marital-status", "occupation", "relationship", "race", "gender", "native-country"]
stages = []

for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]